In [1]:
#!pip install flax diffusers transformers ftfy stable_diffusion_videos yt-dlp -q
import jax; jax.devices()
import random


In [2]:
video_duration = 45 # time in seconds of the video
fps = 30
repeat_prompts_every = 2 # time in seconds of how long to wait between prompts before a different image using the same prompt is generated 
time_per_transition = 0.5 # time in seconds to transition between different prompts

text_transcript = [
      # PUT TEXT TRANSCRIPT HERE
(0, 'Goat'),
(1.68, 'animal'),
(2.16, 'water'),
(3.08, 'goat man'),
(5.2, 'jumping from the city skyline'),
(9.04, 'Jesus Christ'),
(10.68, 'keine'),
(11.38, 'beiden Sprachen'),
(15.2, 'irgendwelchen Worten'),
(19.26, 'Obama'),
(19.86, 'Bush'),
(19.98, 'nein'),

]

In [3]:
prompts = []
num_interpolation_steps = []

for idx, (start, text) in enumerate(text_transcript):
    if idx == len(text_transcript) - 1:
        end = video_duration
    else:
        end = text_transcript[idx+1][0]

    duration = end - start
    prompt_repeats = int((duration - time_per_transition) // repeat_prompts_every)
    remainder_prompt_time = (duration - time_per_transition) % repeat_prompts_every

    prompts += [text] * prompt_repeats
    num_interpolation_steps += [fps * repeat_prompts_every] * prompt_repeats

    if int(remainder_prompt_time * fps) != 0:
        prompts.append(text)
        num_interpolation_steps.append(int(remainder_prompt_time * fps))

    num_interpolation_steps.append(fps//2)
    prompts.append(text)

num_interpolation_steps = num_interpolation_steps[:-1]
num_interpolation_steps[-1] += int(time_per_transition * fps)

seeds = [random.randint(0, 100000) for _ in range(len(prompts))]
print(prompts)
print(num_interpolation_steps)
print(seeds)

['Goat', 'Goat', 'animal', 'animal', 'water', 'water', 'goat man', 'goat man', 'jumping from the city skyline', 'jumping from the city skyline', 'jumping from the city skyline', 'Jesus Christ', 'Jesus Christ', 'keine', 'keine', 'beiden Sprachen', 'beiden Sprachen', 'beiden Sprachen', 'irgendwelchen Worten', 'irgendwelchen Worten', 'irgendwelchen Worten', 'Obama', 'Obama', 'Bush', 'Bush', 'nein', 'nein', 'nein', 'nein', 'nein', 'nein', 'nein', 'nein', 'nein', 'nein', 'nein', 'nein', 'nein', 'nein']
[35, 15, 59, 15, 12, 15, 48, 15, 60, 40, 15, 34, 15, 6, 15, 60, 39, 15, 60, 46, 15, 2, 15, 48, 15, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 60, 30]
[2996, 3007, 89284, 9709, 59780, 92802, 87183, 93793, 31348, 62135, 86347, 21889, 44113, 27862, 86253, 21111, 11172, 61729, 69962, 94569, 86136, 86437, 25667, 59322, 75066, 15852, 9423, 58839, 58756, 37922, 35766, 62233, 66622, 38914, 68764, 79405, 9956, 2345, 98483]


In [5]:
import stable_diffusion_videos
print(dir(stable_diffusion_videos))

from stable_diffusion_videos.pipelines import StableDiffusionWalkPipeline



['FlaxStableDiffusionWalkPipeline', 'Interface', 'RealESRGANModel', 'StableDiffusionWalkPipeline', 'generate_images', 'generate_images_flax', 'get_timesteps_arr', 'make_video_pyav', 'upload_folder_chunked']


In [6]:
from stable_diffusion_videos import StableDiffusionWalkPipeline
from huggingface_hub import hf_hub_download
import torch

pipeline = StableDiffusionWalkPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float16,
).to("cuda")

# Seconds in the song.
audio_offsets = [146, 148]  # [Start, end]
fps = 30  # Use lower values for testing (5 or 10), higher values for better quality (30 or 60)

# Convert seconds to frames
num_interpolation_steps = [(b-a) * fps for a, b in zip(audio_offsets, audio_offsets[1:])]

video_path = pipeline.walk(
    prompts=prompts,
    seeds=seeds,
    num_interpolation_steps=num_interpolation_steps,
    audio_filepath='recorded.mp3',
    audio_start_sec=audio_offsets[0],
    fps=fps,
    height=512,  # use multiples of 64 if > 512. Multiples of 8 if < 512.
    width=512,   # use multiples of 64 if > 512. Multiples of 8 if < 512.
    output_dir='dreams',        # Where images/videos will be saved
    guidance_scale=7.5,         # Higher adheres to prompt more, lower lets model take the wheel
    num_inference_steps=50,     # Number of diffusion steps per image generated. 50 is good default
)

ImportError: cannot import name 'StableDiffusionWalkPipeline' from 'stable_diffusion_videos' (d:\Anaconda\envs\stable_diff\lib\site-packages\stable_diffusion_videos\__init__.py)

In [9]:
import os
import time

tic = time.time()

# Set the output directory to the current working directory
output_dir = os.getcwd()

video_path = pipeline.walk(
    p_params,
    prompts,
    seeds,
    fps=fps,
    num_interpolation_steps=num_interpolation_steps,
    output_dir=output_dir,  # Save the video in the current working directory
    jit=True,
    guidance_scale=7.5,
    num_inference_steps=20,
)

toc = time.time()

print(f"Video saved at: {video_path}")
print(f"Time taken to generate video: {toc - tic} seconds")


NameError: name 'pipeline' is not defined